In [1]:
import board_2d as b2d
import othello as oth
import random as rd
#import carcassonne as carc

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
base_gs = oth.GameState()
base_gs.set_initial_state()
gs = base_gs.duplicate()

In [111]:
gs.make_action(rd.choice(gs.available_actions))
gs.view_game_state()
print(len(gs.available_actions))
print(gs.is_terminal())
print(gs.scores())
print(gs.winner())

   ------------------
7 | O O X X X O O . |
6 | X X O X X X X . |
5 | X X X X O O X O |
4 | O X X X X X X O |
3 | X X X X X X X O |
2 | X . X X X X O O |
1 | X O X X X . . O |
0 | . O . X O . . O |
   ------------------
    0 1 2 3 4 5 6 7 
False
{<Players.PLAYER1: 1>: 65, <Players.PLAYER2: 2>: 48}
None


AttributeError: 'tuple' object has no attribute 'content'

In [4]:
gs.make_action(actions[0])
print(str(gs.board))

Board: (3, 3){'P': <Players.PLAYER1: 1>}(3, 4){'P': <Players.PLAYER2: 2>}(4, 3){'P': <Players.PLAYER1: 1>}(4, 4){'P': <Players.PLAYER1: 1>}(4, 2){'P': <Players.PLAYER1: 1>}


AttributeError: 'tuple' object has no attribute 'coordinates'

In [2]:
board = b2d.Board()
gs = carc.Game_state(board)
gs.set_initial_state()

played
adding (0, 1)  to  (c_val:2_owner:None)
(c_val:2_owner:None)
adding (-1, 0)  to  (p_val:1_owner:None)
(p_val:1_owner:None)
adding (1, 0)  to  (p_val:1_owner:None)
(p_val:1_owner:None)
adding (-1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (0, -1)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (-1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)


In [4]:
#print(gs.board.filled_locations[(0,0)].tile)
for coord,loc in gs.board.available_locations.items():
    print(loc)
    

(-1, 0): {'rd': '(f_val:0_owner:None)', 'r': '(p_val:1_owner:None)', 'ru': '(f_val:0_owner:None)'}
(0, 1): {'d': '(c_val:2_owner:None)'}
(1, 0): {'lu': '(f_val:0_owner:None)', 'l': '(p_val:1_owner:None)', 'ld': '(f_val:0_owner:None)'}
(0, -1): {'u': '(f_val:0_owner:None)'}


In [ ]:
print(gs.board)